# Data vizualisation

In this notebook, we visualize that each step of the data preprocessing was performed correctly.

## Imports

In [1]:
%gui wx
import sys
import os



Gtk-Message: 12:42:29.345: Failed to load module "canberra-gtk-module"


In [2]:
#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"





In [7]:
#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()



['fsl/6.0.7.4', 'freesurfer/7.4.1']

In [8]:
####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

In [9]:
import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

In [10]:
################
# Start FSLeyes 
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

12:44:33: Debug: Adding duplicate image handler for 'Windows bitmap file'
12:44:33: Debug: Adding duplicate animation handler for '1' type
12:44:33: Debug: Adding duplicate animation handler for '2' type
12:44:33: Debug: Adding duplicate image handler for 'Windows bitmap file'
12:44:33: Debug: Adding duplicate animation handler for '1' type
12:44:33: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:11918): Gtk-CRITICAL **: 12:44:34.169: gtk_window_resize: assertion 'height > 0' failed


In [14]:
bids_root = "gdrive_ds000171"
print_dir_tree(bids_root, max_depth=4)

|gdrive_ds000171/
|--- CHANGES
|--- README
|--- T1w.json
|--- dataset_description.json
|--- task-music_bold.json
|--- .ipynb_checkpoints/
|------ README-checkpoint
|------ T1w-checkpoint.json
|------ task-music_bold-checkpoint.json
|--- sub-control01/
|------ anat/
|--------- sub-control01_T1w.nii.gz
|------ func/
|--------- sub-control01_task-music_run-1_bold.nii.gz
|--------- sub-control01_task-music_run-1_events.tsv
|--------- sub-control01_task-music_run-2_bold.nii.gz
|--------- sub-control01_task-music_run-2_events.tsv
|--------- sub-control01_task-music_run-3_bold.nii.gz
|--------- sub-control01_task-music_run-3_events.tsv


In [17]:
preproc_root = "sub_control01_preprocessed"
print_dir_tree(preproc_root, max_depth=2)

|sub_control01_preprocessed/
|--- anat/
|------ T1_to_MNI.mat
|------ T1_to_MNI.nii.gz
|------ sub-control01_T1w_bet.nii.gz
|--- func/
|------ concatenated_standardized.nii.gz
|------ concatenated_standardized_mc.nii.gz
|------ concatenated_standardized_mc.nii.gz.par
|------ concatenated_standardized_mc_coreg.mat
|------ concatenated_standardized_mc_coreg.nii.gz
|------ concatenated_standardized_mc_coreg_fast_wmedge.nii.gz
|------ concatenated_standardized_mc_coreg_fast_wmseg.nii.gz
|------ concatenated_standardized_mc_coreg_highpassed.nii.gz
|------ concatenated_standardized_mc_coreg_highpassed_smoothed.nii.gz
|------ concatenated_standardized_mc_coreg_init.mat
|------ concatenated_standardized_mc.nii.gz.mat/
|------ standardized/


In [18]:
# Set directories
output_dir_anat = 'sub_control01_preprocessed/anat'
output_dir_func = 'sub_control01_preprocessed/func'
subject_id = 'sub-control01'
mni_template = '/cvmfs/neurodesk.ardc.edu.au/containers/fsl_6.0.7.14_20241018/fsl_6.0.7.14_20241018.simg/opt/fsl-6.0.7.14/data/standard/MNI152_T1_1mm.nii.gz'


In [19]:
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data
data = get_json_from_file(op.join(bids_root, 'task-music_bold.json'))
data

{'TaskName': 'Music',
 'RepetitionTime': 3.0,
 'EchoTime': 0.025,
 'FlipAngle': 90.0,
 'SequenceName': 'EPI BOLD',
 'Manufacturer': 'Siemens',
 'ManufacturersModelName': 'Skyra',
 'MagneticFieldStrength': 3.0,
 'ParallelImagingReductionFactor': 0,
 'SliceTimingComputationMethod': '(slicenum*RepetitionTime/TotalSlices), interleaved according to Siemens even-numbered slice convention (even slices first).',
 'SliceTiming': [1.5,
  0.0,
  1.56,
  0.06,
  1.62,
  0.12,
  1.68,
  0.18,
  1.74,
  0.24,
  1.8,
  0.3,
  1.86,
  0.36,
  1.92,
  0.42,
  1.98,
  0.48,
  2.04,
  0.54,
  2.1,
  0.6,
  2.16,
  0.66,
  2.22,
  0.72,
  2.28,
  0.78,
  2.34,
  0.84,
  2.4,
  0.9,
  2.46,
  0.96,
  2.52,
  1.02,
  2.58,
  1.08,
  2.64,
  1.14,
  2.7,
  1.2,
  2.76,
  1.26,
  2.82,
  1.32,
  2.88,
  1.38,
  2.94,
  1.44]}

# 1. Anatomical preprocessing
## 1.1 Vizualisation: anatomical data

In [20]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz'))


(ipykernel_launcher.py:11918): Gdk-WARNING **: 12:55:11.761: gdkdrawable-x11.c:952 drawable is not a pixmap or window


We can observe our first anatomical images: 
* Left: Coronal view
* Center: Transversal view
* Right: Sagittal view

## 1.2 Vizualisation: Skull stripping BET

In [22]:
bet_path = op.join(output_dir_anat, "sub-control01_T1w_bet.nii.gz")
fsleyesDisplay.load(bet_path)

**Observation**: By playing with the layers, we can see that the skull stripping missed some parts.

In [29]:
bet_image = nib.load(bet_path)
bet_image.shape

(256, 256, 176)

### Improving the fit

In [32]:
## Version Lou
data_dir = 'gdrive_ds000171/sub-control01'
anat_file = os.path.join(data_dir, 'anat', f'{subject_id}_T1w.nii.gz')
bet_output_path_adjust = os.path.join(output_dir_anat, f'{subject_id}ADJUSTMENT_T1w_bet.nii.gz')
subprocess.run(f"bet {anat_file} {bet_output_path_adjust} -f 0.45 -g 0" , shell=True)

CompletedProcess(args='bet gdrive_ds000171/sub-control01/anat/sub-control01_T1w.nii.gz sub_control01_preprocessed/anat/sub-control01ADJUSTMENT_T1w_bet.nii.gz -f 0.45 -g 0', returncode=0)

In [34]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz'))
fsleyesDisplay.load(bet_path)
fsleyesDisplay.load(bet_output_path_adjust)

## 1.3.0 MNI

In [27]:
# Version Lab2
from fsl.wrappers import flirt

mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
reference = mni_template
result = op.join(output_dir_anat, 'T1_to_MNI.nii.gz')
target = bet_path

flirt(target, reference, out=result)


Final result: 
0.006220 0.002853 -1.112216 204.408173 
-0.943018 0.533958 0.001633 177.630188 
0.538434 1.083527 -0.014732 -158.840588 
0.000000 0.000000 0.000000 1.000000 



{}

## 1.3.1 Visualisation: MNI template registration (normalization) #step 1.1

In [51]:
mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
reference = mni_template
result = op.join(output_dir_anat, 'T1_to_MNI.nii.gz')

fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(reference) 
fsleyesDisplay.load(result)

# 2. Functionnal preprocessing

## 2.1 Standardize each run

In [41]:
run = 1
std_path = f"standardized/sub-control01_task-music_run-{run}_bold_standardized.nii.gz"

fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(output_dir_func, std_path))

In [43]:
# Check image shape
std_image = nib.load(op.join(output_dir_func, std_path))
std_image.shape

(80, 80, 50, 105)

## 2.2 Concatenate the 3 runs

In [44]:
concat_output_path = os.path.join(output_dir_func, 'concatenated_standardized.nii.gz')


fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(concat_output_path)

## 2.3 Motion correction # step 4

In [52]:
mc_output_path = os.path.join(output_dir_func, 'concatenated_standardized_mc.nii.gz')

fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(mc_output_path)

In [59]:
mc_image = nib.load(mc_output_path)
mc_image.shape

(80, 80, 50, 315)

In [ ]:
from helper import *
import numpy as np
import matplotlib.pyplot as plt

mots= load_mot_params_fsl_6_dof('sub_control01_preprocessed/func/concatenated_standardized_mc.nii.gz.par')
fd=compute_FD_power(mots).to_numpy()
threshold = 1 

plt.plot(list(range(1,fd.size+1)),fd)
plt.xlabel('Volume')
plt.ylabel('FD displacement (mm)')
plt.hlines(threshold, 0,320,colors='black', linestyles='dashed', label= 'FD limit')
plt.legend()
plt.show()

## 2.4 Frequency spectrum

In [ ]:

from scipy.fft import fft, fftfreq

img = nib.load('sub_control01_preprocessed/func/concatenated_standardized_mc.nii.gz')
data = img.get_fdata()
mean_time_series = data.mean(axis=(0, 1, 2))
n_timepoints = mean_time_series.shape[0]
sampling_freq = 1/3  

# Compute the FFT of the mean time series
fft_values = fft(mean_time_series)
frequencies = fftfreq(n_timepoints, d=1/sampling_freq)

power_spectrum = np.abs(fft_values) ** 2
# Only take the positive half of the frequencies and power spectrum
positive_frequencies = frequencies[:n_timepoints // 2]
positive_power_spectrum = power_spectrum[:n_timepoints // 2]

plt.figure(figsize=(10, 6))
plt.plot(positive_frequencies, positive_power_spectrum)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.title('Power Spectrum of fMRI Time Series')
plt.show()
